In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tweepy==4.10.1
import tweepy
import datetime
import time
try:
  from GoogleNews import GoogleNews
except:
  !pip install GoogleNews
  from GoogleNews import GoogleNews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 292 kB 8.0 MB/s 


In [ ]:
try:
  from transformers import BertTokenizer, BertModel
except:
  !pip install transformers
  from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import tqdm
import json
import os
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.0 MB/s 
     |████████████████████████████████| 7.6 MB 32.2 MB/s 
     |████████████████████████████████| 182 kB 39.6 MB/s 


#Getting Data from API

In [ ]:
def clean_results(res, name, keys):
    for d in res:
        d['text'] = d['title']+'. '+d['desc']
        d['date'] = [d['date'], d['datetime']]
        for k in keys:
            del d[k]
    return res

def getResponses(query):

    consumer_key = "imwrWfsSa6ClKDtco1m5aamlL"
    consumer_secret = "Ee1l74nk8Sx9a3lgYRGRtIMsA9w9CJp6UWS5o5Hujaktb9FUgh"
    bearer_token = "AAAAAAAAAAAAAAAAAAAAANxGjQEAAAAAJRz42cJaTM0Vjftlj5qBXZbFUoQ%3Ds6sj59WRac4zF8fAn973cVazbok2TMSfcsryTIfBXVF4NERfXg"

    access_token = "1591903520805961728-idXQZQJ5ETnqmTxG1MrKepmW1pWT5Z"
    access_token_secret = "oaa6utpuNwrTFfryWg2M5BQvrSN3FoKxa3XxfqIC7Z1nB"

    auth = tweepy.auth.OAuthHandler(
        consumer_key, consumer_secret#, access_token, access_token_secret
    )
    auth.set_access_token(access_token, access_token_secret)

    # api = tweepy.API(auth)
    client = tweepy.Client(bearer_token=bearer_token)

    googlenews = GoogleNews(period='1month', lang='en', region='US')

    responses = []
    googleData = []
    del_keys = ['media', 'img', 'datetime', 'title', 'desc']
    
    

    responses.append([client.search_all_tweets(query + " lang:en -is:retweet -links -https", max_results=20, tweet_fields = 
                            ['created_at', 'geo', 'public_metrics', 'text','entities',],)])
    googlenews.search(query)
    for i in range(2):
        result = googlenews.page_at(i+1)
        googleData.extend(clean_results(result, query, del_keys))
    googlenews.clear()

    return responses , googleData

def getData(queries):
    responses, googleData = getResponses(queries)
    data_tweet = [[ "Text", "URL"]]
    data_google = [[ "Text", "URL"]]
    for response in responses:
        tweets = response[0].data
        for tweet in tweets:
            data_tweet.append([tweet.text, "twitter"])

    for gdata in googleData:
        data_google.append([gdata['text'], gdata['link']])
    return data_tweet, data_google

#Getting Scores from Model

In [ ]:
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

CKPT_PATH = "/content/drive/Shareddrives/SI650/model_3layer_ckpt.pt"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', padding=True, truncation=True)
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)


def get_embeddings(tokenizer, bert_model, text_list):
  # text = "Here is the sentence I want embeddings for."
  sentence_embeddings = torch.zeros(len(text_list),768)
  for i in range(len(text_list)):
    marked_text = "[CLS] " + text_list[i][:512] + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens]).to(DEVICE)
    segments_tensors = torch.tensor([segments_ids]).to(DEVICE)
    # print(tokens_tensor.shape, segments_tensors.shape)
    with torch.no_grad():
      outputs = bert_model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]
    # token_embeddings = torch.stack(hidden_states, dim=0)
    # token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs = hidden_states[-2][0]
    sentence_embeddings[i,:] = torch.mean(token_vecs,dim=0)
  return sentence_embeddings

class StockRelevanceModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.classifier = nn.Sequential(
        nn.Linear(768, 1024),
        nn.BatchNorm1d(1024),
        nn.Dropout(p=0.5),
        nn.ReLU(),
        nn.Linear(1024,256),
        nn.BatchNorm1d(256),
        nn.Dropout(p=0.5),
        nn.ReLU(),
        nn.Linear(256, 4)
    )

  def forward(self, x):
    return self.classifier(x)

def get_rel_scores(data, model=None):
    if not model:
      model = StockRelevanceModel()
      model = torch.load(CKPT_PATH, map_location=torch.device(DEVICE))
    model.eval()
    bert_model.to(DEVICE)
    labels = torch.tensor([[0,1,2,3]], dtype=torch.float32).to(DEVICE)
    scores = []
    # for text in data:
    #     print(text)
    sentence_embeddings = get_embeddings(tokenizer, bert_model, data).to(DEVICE)
    logits = model(sentence_embeddings)
    prob = nn.functional.softmax(logits, dim=1)
    avg_label = (prob@labels.T).squeeze()
    scores = avg_label.cpu().detach().tolist()
    return scores

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
QE_PATH = "/content/drive/Shareddrives/SI650/comp_qe.json"
with open(QE_PATH, 'r') as f:
    qe_dict = json.load(f)

query = 'Apple'

if query.lower() in qe_dict.keys():
    query_exp = qe_dict[query.lower()]
else:
    query_exp = qe_dict['other']
print("Getting data from API...")
data_tweet, data_google = getData(' '.join(query_exp.split(' ')[:2]))

print("Converting data to DataFrames")
tweet_df = pd.DataFrame(data_tweet[1:], columns=data_tweet[0])
google_df = pd.DataFrame(data_google[1:], columns=data_google[0])
# scores = get_rel_scores(tweet_df['Text'])
# tweet_df['scores'] = scores
tweet_df = tweet_df.drop_duplicates(subset=['Text'])
google_df = google_df.drop_duplicates(subset=['Text'])
tweet_scores = get_rel_scores(list(tweet_df['Text']))
google_scores = get_rel_scores(list(google_df['Text']))

Getting data from API...
Converting data to DataFrames


In [ ]:
# len(tweet_df.drop_duplicates(subset=['Text']))
tweet_df.iloc[np.argsort(tweet_scores)[::-1]].head()

,Text,URL
6,SHARES OF SOME BIG TECH COS RISE PREMARKET AFT...,twitter
9,SHARES OF SOME BIG TECH COS RISE PREMARKET AFT...,twitter
8,Shares Of Some Big Tech Cos Rise Premarket Aft...,twitter
12,Sold my Visa $V position at a ~30% return excl...,twitter
7,*$AAPL $META $GOOGL $MSFT $TSLA** - SHARES OF ...,twitter


In [ ]:
google_df.iloc[np.argsort(google_scores)[::-1]][['Text','URL']].head()

,Text,URL
14,Berkshire Hathaway Inc. Cl B stock outperforms...,https://www.msn.com/en-us/money/topstocks/berk...
1,Berkshire Hathaway Inc. Cl B stock outperforms...,https://www.marketwatch.com/story/berkshire-ha...
8,Is Warren Buffett's Berkshire Hathaway Stock a...,https://www.fool.com/investing/2022/12/16/is-w...
10,5 Phenomenal Stocks in Warren Buffett's Secret...,https://www.fool.com/investing/2022/12/16/5-st...
19,Berkshire Hathaway HomeServices Expands Presen...,https://www.businesswire.com/news/home/2022121...


In [ ]:
google_df['Scores'] = google_scores

In [ ]:
data_google

[['Text', 'URL']]